In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [5]:
# Loading Excel file to get sheet names

sheets1 = pd.ExcelFile('../data/raw/Public-School-Data-Files-2020_2021-School-Year.xls')
print(sheets1.sheet_names)

# Loading one dataframe to get column names

nov22 = pd.read_excel('../data/raw/Public-School-Data-Files-2020_2021-School-Year.xls', sheet_name="Campus Report_November 22", header=5)
print(nov22.head(20))

['Dashboard', 'Campus Report_August 1', 'Campus Report_July 25', 'Campus Report_July 18', 'Campus Report_July 11', 'Campus Report_July 04', 'Campus Report_June 27', 'Campus Report_June 20', 'Campus Report_June13', 'Campus Report_June 6', 'Campus Report_May 30', 'Campus Report_May 23', 'Campus Report_May 16', 'Campus Report_May 09', 'Campus Report_May 02', 'Campus Report_April 25', 'Campus Report_April 18', 'Campus Report_April 11', 'Campus Report_April 4', 'Campus Report_March 28', 'Campus Report_March 21', 'Campus Report_March 14', 'Campus Report_March 7', 'Campus Report_February 28', 'Campus Report_February 21', 'Campus Report_February 14', 'Campus Report_February 7', 'Campus Report_January 31', 'Campus Report_January 24', 'Campus Report_January 17', 'Campus Report_January 12', 'Campus Report_January 5th', 'Campus Report_December 29', 'Campus Report_December 20', 'Campus Report_December 15', 'Campus Report_December 6', 'Campus Report_November 29', 'Campus Report_November 22', 'Distri

In [2]:
# List of sheet names from original Excel file (excludes district reports, as this analysis will focus
# on the campus level).

sheets = ['Campus Report_August 1', 'Campus Report_July 25', 'Campus Report_July 18', 'Campus Report_July 11', 
          'Campus Report_July 04', 'Campus Report_June 27', 'Campus Report_June 20', 'Campus Report_June13', 
          'Campus Report_June 6', 'Campus Report_May 30', 'Campus Report_May 23', 'Campus Report_May 16', 
          'Campus Report_May 09', 'Campus Report_May 02', 'Campus Report_April 25', 'Campus Report_April 18', 
          'Campus Report_April 11', 'Campus Report_April 4', 'Campus Report_March 28', 'Campus Report_March 21', 
          'Campus Report_March 14', 'Campus Report_March 7', 'Campus Report_February 28', 
          'Campus Report_February 21', 'Campus Report_February 14', 'Campus Report_February 7', 
          'Campus Report_January 31', 'Campus Report_January 24', 'Campus Report_January 17', 
          'Campus Report_January 12', 'Campus Report_January 5th', 'Campus Report_December 29', 
          'Campus Report_December 20', 'Campus Report_December 15', 'Campus Report_December 6', 
          'Campus Report_November 29']

# Reversing list to get reports in chronological order from beginning of school year

sheets.reverse()

# Getting first weekly report as base dataframe (November 22 is not in the list above)

df = pd.read_excel('../data/raw/Public-School-Data-Files-2020_2021-School-Year.xls', sheet_name="Campus Report_November 22", header=5)

# Renaming columns to remove line breaks and shorten column names

df.rename(columns = {'District Name':'District', 'District\nLEA\nNumber':'Dist LEA', 
                    'Total District\nEnrollment as\nof October 30, 2020':'Dist Enrollment 10/30/20', 
                    'Approximate\nDistrict On Campus\nEnrollment as of\nOctober 30, 2020':'Dist On-Campus Enrollment 10/30/20', 
                    'Total School\nEnrollment as\nof October 30, 2020':'Sch Enrollment 10/30/2020', 
                    'On-Campus\nEnrollment for\nSchool as of\nOctober 30, 2020':'Sch On-Campus Enrollment 10/30/2020', 
                    'Campus\nID':'Campus ID', 'Total\nStudent\nCases':'Nov22', 
                    'Total\nStaff\nCases':'Total staff cases_Nov22'},
                     inplace=True)

# Replace 'multiple campus' listings with NaN campus ID
    
df['Campus ID'].replace('Multiple\nCampus', np.nan, inplace=True)
df['Campus ID'] = df['Campus ID'].str.strip("'").astype('float')
    
# Convert asterisks (suppressed values -- see below) to NaN, add to new dataframe
# Using float so NaNs can coexist in column
    
df['Nov22'].replace([r'*', ' '], np.nan, inplace=True) 
df['Nov22'] = df['Nov22'].astype('float')

df.dropna(subset='Campus ID', inplace=True)
df['Campus ID'] = df['Campus ID'].astype('int')

In [ ]:
# Empty list for adding date abbreviations (to be appended to column names)

abbrevs=[]

# List for raw dataframes

ds = []

for i, sheet in enumerate(sheets):
    
    # Loading sheets into dataframes in list
    
    ds.append(pd.read_excel('../data/raw/Public-School-Data-Files-2020_2021-School-Year.xls', 
                                                sheet_name = sheet, header=5))
    
    # Create date abbreviation for student cases columns
    # sheet[14:17] gets first three letters of month
    # sheet[-1] and [-2:] get numerical date of one or two
    # digits
    
    if sheet[-2] == " ":
        abbrev = sheet[14:17] + sheet[-1]
    else:
        abbrev = sheet[14:17] + sheet[-2:]
        
    # One sheet has "5th" instead of "5", this fixes that problem
        
    if abbrev == 'Janth':
        abbrev = "Jan5" 
    
    # Add abbreviations to list
    
    abbrevs.append(abbrev)
    
    # Renaming student total column (used columns.values becase not all
    # sheets use the same column headers)
    
    ds[i].rename(columns = {'Campus\nID': 'Campus ID',
                            ds[i].columns.values[13]: abbrevs[i]}, inplace=True)  
    
    # Cutting each dataframe down to only campus ID and student cases                        
                            
    ds[i] = ds[i][['Campus ID', abbrevs[i]]]
                            
    # Replace 'multiple campus' listings with NaN campus ID and drop rows with
    # no campus ID
    
    ds[i]['Campus ID'].replace('Multiple\nCampus', np.nan, inplace=True)
    ds[i].dropna(subset='Campus ID', inplace=True)
                            
    # Remove leading apostrophe and convert to int
                            
    ds[i]['Campus ID'] = ds[i]['Campus ID'].str.strip("'").astype('int')

    # Convert asterisks (suppressed values) and blank spaces to NaN
    # Using float again so NaNs can coexist in column
    
    ds[i][abbrevs[i]].replace([r'*', ' '], np.nan, inplace=True)  
    ds[i][abbrevs[i]] = ds[i][abbrevs[i]].astype('float')
    
    # One campus has duplicated reports (can detail this in report later),
    # and it looks like like the second report is more likely to be accurate.
    
    ds[i].drop_duplicates(subset='Campus ID', keep='last', inplace=True)
    
    # Merge all data frames to base dataframe
    
    df = df.merge(ds[i], how='inner', on='Campus ID')

In [ ]:
# Note: Looks like most schools have the same number of reports
# in the last 4–5 reports, so I thought I might have some sort of
# error, but I did find one campus (index 1185) with different
# numbers in Jul18 and Jul25, so I think this probably worked.

# Mean cases goes up every week (just barely)

print(df.sample(20)['Campus ID'])

In [ ]:
def pullstats(campusid):
    
    '''Grabs percentage of economically disadvantaged students, average class size on campus,
    STAAR performance rates for 'meets grade level or above', and total operating expenditures
    per student.'''
    
    campus_stats = pd.DataFrame({'Campus ID': pd.Series(dtype='int'), 
                                 'Econ disadv': pd.Series(dtype='float'),
                                 'Avg class': pd.Series(dtype='float'),
                                 'STAAR 2021': pd.Series(dtype='float'),
                                 'Spending': pd.Series(dtype='float')})

    campus_stats['Campus ID'] = [campusid]
                                 
    # Request TEA school report card corresponding to campus ID
    
    r = requests.get(f'https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=0&ccyy=2022&lev=C&id={str(campusid).zfill(9)}&prgopt=reports%2Fsrc%2Fsrc.sas')
    
    # Parse HTML
    
    soup = BeautifulSoup(r.text, 'html.parser')

    # Find percentage of economically disadvantaged students on campus
    # Note: state average 60.7%

    try:
        econ = soup.find('td', string=re.compile('Economically Disadvantaged'))

        for i in range(3):
            econ = econ.next_element
    
        campus_stats['Econ disadv'] = [pd.to_numeric(econ.get_text(strip=True).strip('%')) / 100]
        
    except AttributeError:
        
        campus_stats['Econ disadv'] = np.nan

    try:  
        # Gets average class size on campus, only taking into account listed values
        # Note: state averages listed by class

        classes = ['Kindergarten', 'Grade 1', 'Grade 2', 'Grade 3', 'Grade 4', 'Grade 5', 'Grade 6', 
                  'English/Language Arts', 'Foreign Languages', 'Mathematics', 'Science', 'Social Studies']
        class_sizes = []

        for cl in classes:
            size = soup.find('td', string=re.compile(cl))

            # Not totally sure why, but the campus number is three elements away

            for i in range (3):
                size = size.next_element

            class_sizes.append(pd.to_numeric(size.text, errors='coerce'))

        # np.nanmean ignores NaNs and calculates mean of numbers

        campus_stats['Avg class'] = [np.nanmean(class_sizes)]
        
    except (AttributeError, IndexError):
        
        campus_stats['Avg class'] = np.nan

        # Get "STAAR Performance Rates at Meets Grade Level or Above" from all subjects in 2021
        # Note: 2021 is closer to start of pandemic and infection stats (double-check?)
        # Note: actual figures from 2019 are in PDF form
    try:
        staar = soup.find_all('th', string=re.compile('2021'))[5]

        for i in range(6):
            staar = staar.next_sibling

        campus_stats['STAAR 2021'] = [pd.to_numeric((staar.get_text(strip=True).rstrip('%')), errors='coerce') / 100]

    except (AttributeError, IndexError):
        
        campus_stats['STAAR 2021'] = np.nan
        
    try:
        # Expenditures per student

        exp = soup.find('td', string=re.compile('Total Operating Expenditures'))

        for i in range(2):
            exp = exp.next_sibling

        campus_stats['Spending'] = [pd.to_numeric((exp.get_text(strip=True)).replace(',', '').strip('$'), errors='coerce')]
        
    except AttributeError:
        
        campus_stats['Spending'] = np.nan

    return campus_stats

In [ ]:
# DON'T RUN THIS AGAIN it takes ages

#school_stats = pd.DataFrame()

#for campus in df['Campus ID']:
#    school = pullstats(campus)
#    school_stats = pd.concat([school_stats, school])
    
#school_stats.to_csv('../data/school_stat.csv')

In [19]:
schoolstats = pd.read_csv('../data/raw/school_stat.csv', usecols=['Campus ID', 'Econ disadv', 'Avg class', 'STAAR 2021', 'Spending'])

In [20]:
df = df.merge(schoolstats, on='Campus ID', how='inner')

df.to_csv('../data/interim/df.csv')

df.head()

,District,Dist LEA,Dist Enrollment 10/30/20,Dist On-Campus Enrollment 10/30/20,Campus Name,Campus ID,Sch Enrollment 10/30/2020,Sch On-Campus Enrollment 10/30/2020,New\nStudent\nCases,New\nStaff\nCases,...,Jun27,Jul04,Jul11,Jul18,Jul25,Aug1,Econ disadv,Avg class,STAAR 2021,Spending
0,CAYUGA ISD,'001902,535,489,CAYUGA EL,1902103,236,208,NaN,NaN,...,11.0,11.0,11.0,11.0,11.0,11.0,0.459,15.833333,0.68,6919.0
1,CAYUGA ISD,'001902,535,489,CAYUGA MIDDLE,1902041,133,126,NaN,NaN,...,11.0,11.0,11.0,11.0,11.0,11.0,0.396,15.583333,0.52,8476.0
2,CAYUGA ISD,'001902,535,489,CAYUGA H S,1902001,166,155,NaN,NaN,...,16.0,16.0,16.0,16.0,16.0,16.0,0.343,9.820000,0.72,10656.0
3,ELKHART ISD,'001903,1200,1092,ELKHART DAEP,1903002,NR,NR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ELKHART ISD,'001903,1200,1092,ELKHART H S,1903001,344,310,NaN,NaN,...,21.0,21.0,21.0,21.0,21.0,21.0,0.345,9.300000,0.66,11177.0


In [44]:
df = pd.read_csv('../data/interim/df.csv')

In [46]:
df.columns

Index(['Unnamed: 0', 'District', 'Dist LEA', 'Dist Enrollment 10/30/20',
       'Dist On-Campus Enrollment 10/30/20', 'Campus Name', 'Campus ID',
       'Sch Enrollment 10/30/2020', 'Sch On-Campus Enrollment 10/30/2020',
       'New\nStudent\nCases', 'New\nStaff\nCases', 'On\nCampus', 'Off\nCampus',
       'Unknown', 'Nov22', 'Total staff cases_Nov22', 'On\nCampus.1',
       'Off\nCampus.1', 'Unknown.1', 'Nov29', 'Dec6', 'Dec15', 'Dec20',
       'Dec29', 'Jan5', 'Jan12', 'Jan17', 'Jan24', 'Jan31', 'Feb7', 'Feb14',
       'Feb21', 'Feb28', 'Mar7', 'Mar14', 'Mar21', 'Mar28', 'Apr4', 'Apr11',
       'Apr18', 'Apr25', 'May02', 'May09', 'May16', 'May23', 'May30', 'Jun6',
       'Jun13', 'Jun20', 'Jun27', 'Jul04', 'Jul11', 'Jul18', 'Jul25', 'Aug1',
       'Econ disadv', 'Avg class', 'STAAR 2021', 'Spending'],
      dtype='object')

In [51]:
df.drop(['New\nStudent\nCases', 'New\nStaff\nCases', 'Unknown', 'Unnamed: 0', 'On\nCampus', 'Off\nCampus', 
        'On\nCampus.1', 'Off\nCampus.1', 'Unknown.1'], axis=1, inplace=True)

KeyError: "['New\\nStudent\\nCases', 'New\\nStaff\\nCases', 'Unknown', 'Unnamed: 0', 'On\\nCampus', 'Off\\nCampus', 'On\\nCampus.1', 'Off\\nCampus.1', 'Unknown.1'] not found in axis"

In [50]:
df.columns

Index(['District', 'Dist LEA', 'Dist Enrollment 10/30/20',
       'Dist On-Campus Enrollment 10/30/20', 'Campus Name', 'Campus ID',
       'Sch Enrollment 10/30/2020', 'Sch On-Campus Enrollment 10/30/2020',
       'Nov22', 'Total staff cases_Nov22', 'Nov29', 'Dec6', 'Dec15', 'Dec20',
       'Dec29', 'Jan5', 'Jan12', 'Jan17', 'Jan24', 'Jan31', 'Feb7', 'Feb14',
       'Feb21', 'Feb28', 'Mar7', 'Mar14', 'Mar21', 'Mar28', 'Apr4', 'Apr11',
       'Apr18', 'Apr25', 'May02', 'May09', 'May16', 'May23', 'May30', 'Jun6',
       'Jun13', 'Jun20', 'Jun27', 'Jul04', 'Jul11', 'Jul18', 'Jul25', 'Aug1',
       'Econ disadv', 'Avg class', 'STAAR 2021', 'Spending'],
      dtype='object')

In [53]:
df.drop('Total staff cases_Nov22', axis=1, inplace=True)

In [54]:
df.columns

Index(['District', 'Dist LEA', 'Dist Enrollment 10/30/20',
       'Dist On-Campus Enrollment 10/30/20', 'Campus Name', 'Campus ID',
       'Sch Enrollment 10/30/2020', 'Sch On-Campus Enrollment 10/30/2020',
       'Nov22', 'Nov29', 'Dec6', 'Dec15', 'Dec20', 'Dec29', 'Jan5', 'Jan12',
       'Jan17', 'Jan24', 'Jan31', 'Feb7', 'Feb14', 'Feb21', 'Feb28', 'Mar7',
       'Mar14', 'Mar21', 'Mar28', 'Apr4', 'Apr11', 'Apr18', 'Apr25', 'May02',
       'May09', 'May16', 'May23', 'May30', 'Jun6', 'Jun13', 'Jun20', 'Jun27',
       'Jul04', 'Jul11', 'Jul18', 'Jul25', 'Aug1', 'Econ disadv', 'Avg class',
       'STAAR 2021', 'Spending'],
      dtype='object')

## New loop for converting infection numbers to weekly changes

In [65]:
dates = ['Nov22', 'Nov29', 'Dec6', 'Dec15', 'Dec20', 'Dec29', 'Jan5', 'Jan12',
       'Jan17', 'Jan24', 'Jan31', 'Feb7', 'Feb14', 'Feb21', 'Feb28', 'Mar7',
       'Mar14', 'Mar21', 'Mar28', 'Apr4', 'Apr11', 'Apr18', 'Apr25', 'May02',
       'May09', 'May16', 'May23', 'May30', 'Jun6', 'Jun13', 'Jun20', 'Jun27',
       'Jul04', 'Jul11', 'Jul18', 'Jul25', 'Aug1']

# Upom -- does the loop below look right to you?
# Is this actually calculating what I think it is?

for i in range(len(dates)):
    if i == 0:
        df[dates[i]+'_change'] = 0
    else:
        df[dates[i]+'_change'] = df[dates[i]] - df[dates[i-1]]

df.columns

Index(['District', 'Dist LEA', 'Dist Enrollment 10/30/20',
       'Dist On-Campus Enrollment 10/30/20', 'Campus Name', 'Campus ID',
       'Sch Enrollment 10/30/2020', 'Sch On-Campus Enrollment 10/30/2020',
       'Nov22', 'Nov29', 'Dec6', 'Dec15', 'Dec20', 'Dec29', 'Jan5', 'Jan12',
       'Jan17', 'Jan24', 'Jan31', 'Feb7', 'Feb14', 'Feb21', 'Feb28', 'Mar7',
       'Mar14', 'Mar21', 'Mar28', 'Apr4', 'Apr11', 'Apr18', 'Apr25', 'May02',
       'May09', 'May16', 'May23', 'May30', 'Jun6', 'Jun13', 'Jun20', 'Jun27',
       'Jul04', 'Jul11', 'Jul18', 'Jul25', 'Aug1', 'Econ disadv', 'Avg class',
       'STAAR 2021', 'Spending', 'Nov22_change', 'Nov29_change', 'Dec6_change',
       'Dec15_change', 'Dec20_change', 'Dec29_change', 'Jan5_change',
       'Jan12_change', 'Jan17_change', 'Jan24_change', 'Jan31_change',
       'Feb7_change', 'Feb14_change', 'Feb21_change', 'Feb28_change',
       'Mar7_change', 'Mar14_change', 'Mar21_change', 'Mar28_change',
       'Apr4_change', 'Apr11_change', 'Apr1

In [67]:
# November 22 is all zeroes so it serves as a baseline.
# I also had the idea that it could report infection numbers and serve as a 
# predictor variable, as that might be useful. What do you think?
# I'm not very familiar with time-series forecasting (if that's what
# you have in mind for this -- I'm not totally sure) so I don't know
# if that seems reasonable.

df['Nov22_change'].head(25)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
Name: Nov22_change, dtype: int64

In [68]:
# It looks like this is showing both zeroes and NaNs. I assume that means if there's
# no change, it shows up as zero, but if there's a NaN in either column, it'll show up as
# NaN. That seems reasonable to me, though there could probably be an argument made for
# two suppressed values getting a "0" change instead of a NaN. That would require refiguring
# a some code above, though, as well.

df['Nov29_change'].head(25)

0     0.0
1     2.0
2     0.0
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    0.0
11    NaN
12    NaN
13    NaN
14    0.0
15    0.0
16    0.0
17    NaN
18    0.0
19    NaN
20    0.0
21    0.0
22    0.0
23    NaN
24    NaN
Name: Nov29_change, dtype: float64

In [70]:
df['Dec6'].head(25)

0      7.0
1      9.0
2     13.0
3      NaN
4      5.0
5      NaN
6      NaN
7      NaN
8      NaN
9      5.0
10    14.0
11     NaN
12     NaN
13     NaN
14     0.0
15     0.0
16     0.0
17     NaN
18    30.0
19     NaN
20     5.0
21    12.0
22    19.0
23     NaN
24     NaN
Name: Dec6, dtype: float64

In [77]:
df['Aug1'].isna().sum()

3693

## Notes

"* indicates cell has been suppressed, a blank cell indicates no report has been received for a given district in the indicated time period, a 0 indicates that a report was received and no cases were reported for that group in the reported time period. Single-campus student cases and sources of infection are suppressed when (1) reported student cases are fewer than 5, (2) a campus has at least a 90% student positivity rate when on-campus enrollment for a school is at least 15 students, or (3) a campus has at least a 50% positivity rate when on-campus enrollment has fewer than 15 students. If only one campus in a district has suppressed student numbers then student and source of infection numbers for the campus with the next smallest numbers of positive students are also suppressed. Cumulative student cases and sources of infection numbers for a campus are suppressed when (1) student cases are less than five, or (2) current report numbers have been suppressed for the first three weeks that student cases are reported. If there is only one campus reporting in a district and it is a multiple campus, student and source of infection numbers are not suppressed for the district total. Otherwise, district totals are suppressed when (1) student cases are fewer than 5, or (2) a district has at least a 90% student positivity rate when total district enrollment is least 15 students, or, (3) a district has at least a 50% positivity rate when total district enrollment has fewer than 15 students, or (4) cases on a campus have been suppressed for the first three weeks that student cases are reported and there are fewer than 5 campuses reporting in a district."